In [1]:
########to load landmarks for face###############
import bz2
import os

from urllib.request import urlopen

def download_landmarks(dst_file):
    url = 'http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2'
    decompressor = bz2.BZ2Decompressor()
    
    with urlopen(url) as src, open(dst_file, 'wb') as dst:
        data = src.read(1024)
        while len(data) > 0:
            dst.write(decompressor.decompress(data))
            data = src.read(1024)

dst_dir = 'F:/face2/face-recognition-master/models'
dst_file = os.path.join(dst_dir, 'landmarks.dat')

if not os.path.exists(dst_file):
    os.makedirs(dst_dir)
    download_landmarks(dst_file)

In [2]:
#########creating model ############
from model import create_model

nn4_small2 = create_model()

Using TensorFlow backend.


In [3]:
############triplet loss computation and add it as a layer to the model nn4###############
from keras import backend as K
from keras.models import Model
from keras.layers import Input, Layer

# Input for anchor, positive and negative images
in_a = Input(shape=(96, 96, 3))
in_p = Input(shape=(96, 96, 3))
in_n = Input(shape=(96, 96, 3))

# Output for anchor, positive and negative embedding vectors
# The nn4_small model instance is shared (Siamese network)
emb_a = nn4_small2(in_a)
emb_p = nn4_small2(in_p)
emb_n = nn4_small2(in_n)

class TripletLossLayer(Layer):
    def __init__(self, alpha, **kwargs):
        self.alpha = alpha
        super(TripletLossLayer, self).__init__(**kwargs)
    
    def triplet_loss(self, inputs):
        a, p, n = inputs
        p_dist = K.sum(K.square(a-p), axis=-1)
        n_dist = K.sum(K.square(a-n), axis=-1)
        return K.sum(K.maximum(p_dist - n_dist + self.alpha, 0), axis=0)
    
    def call(self, inputs):
        loss = self.triplet_loss(inputs)
        self.add_loss(loss)
        return loss

# Layer that computes the triplet loss from anchor, positive and negative embedding vectors
triplet_loss_layer = TripletLossLayer(alpha=0.2, name='triplet_loss_layer')([emb_a, emb_p, emb_n])

# Model that can be trained with anchor, positive negative images
nn4_small2_train = Model([in_a, in_p, in_n], triplet_loss_layer)

In [4]:
########using the nn4 pretrained model,its basically a inception model##########
nn4_small2_pretrained = create_model()
nn4_small2_pretrained.load_weights('F:/face2/face-recognition-master/weights/nn4.small2.v1.h5')#load the weights for the model

In [5]:
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from align import AlignDlib

%matplotlib inline

def load_image(path):
    img = cv2.imread(path, 1)
    # OpenCV loads images with color channels
    # in BGR order. So we need to reverse them
    #return img[...,::-1]
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    
    return img

# Initialize the OpenFace face alignment utility
alignment = AlignDlib('F:/face2/face-recognition-master/models/landmarks.dat') ###for aligning the face using dlib



In [6]:
###return aligned image
def align_image(img):
    return alignment.align(96, img, alignment.getLargestFaceBoundingBox(img), 
                           landmarkIndices=AlignDlib.OUTER_EYES_AND_NOSE)

In [7]:
##############run from here#####################################################################
#######################################################3
##################################333321

In [14]:
#########take the live feed input and also to extract embeddings from the live feed of training image########
from tkinter import *

import numpy as np
import cv2

import tkinter
#window=tkinter.Tk()
good_image_indx = []
unfit_image_indx = []

count=0
cc=0

user=[]
embedded = np.zeros((100, 128)) ####the shape is 100,.. beacuse 10 images for one person so lenght needed is 10 for one person ,like 128 embeddings of each person will be extracted from the 10 images of the same person i.e 10 frames of the vedio,so here 100 meaning we can feed 10 people at a time and extract their embeddings and store it into this array
cap = cv2.VideoCapture(0)

while(True):
    ret, frame = cap.read()

    # Our operations on the frame come here
    imgf = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    if cc==10:###wen the 10 images or 10 frames of one person is done then the vedio wil stop for a delay of 15 secs,i.e it wont extract any embeddings from the vedio,and after 15secs the text appears to change the person,and the next person shuld appear in front of camera and then after 10 frames again the vedio will pause and this can continue for how many ever people u need ,but the embeddings can be stored only for the first 10 people
        cv2.putText(img = imgf, text = "change", org = (21,25), fontFace = cv2.FONT_HERSHEY_DUPLEX, fontScale = 1, 
                        color = (0, 255, 0))
        def show_entry_fields():
            print("First Name: %s\nLast Name: %s" % (e1.get(), e2.get()))

        master = tkinter.Tk()
        master.geometry('360x100')
        def whenpressed():
            user.append(e1.get())
            master.destroy()
        master.title("Face_Recognition")
        Label(master, text="Please enter your name and press quit").grid(row=3,column=1)
        Label(master, text="Face Recognition", fg = "white",bg = "black",font = "Helvetica 14 bold italic").grid(row=2,column=1)

        e1 = Entry(master)
        #e2 = Entry(master)

        e1.grid(row=4, column=1)
        #e2.grid(row=1, column=1)
        print(user)
        #print(e1.get())
        Button(master, text='Quit', width=50,command=whenpressed).grid(row=5, column=1, sticky=W, pady=4)
        #Button(master, text='Show', command=show_entry_fields).grid(row=3, column=1, sticky=W, pady=4)

        mainloop( )

        #res=Label(master)
        cv2.putText(img = imgf, text = "changexbro", org = (45,48), fontFace = cv2.FONT_HERSHEY_DUPLEX, fontScale = 1, 
                        color = (0, 255, 0)) ###this line is redundant
        key = cv2.waitKey(10000)
        cc=0 ##here cc is again made 0 for the next person , till cc reaches 9 the below whole process of embedding extraction keeps happening for all the 10 frames of the person
    # Capture frame-by-frame

    img = align_image(imgf) ##call the align image function for extracting the aligned face for the person
    try:
        img = (img / 255.).astype(np.float32)
        embedded[count] = nn4_small2_pretrained.predict(np.expand_dims(img, axis=0))[0]### here embeddings are stored for all the people ,first 10 are for the first person ,next 10 for the next person and so on
        cc=cc+1###here cc is incremented , it is incremented only wen the embedding is extracted ,nt wen it is not clear to extract,this is done to have only 10 proper embeddings and not the error
        count+=1## this is never made 0 , it is for incremneting for all the people like if it is 4 peopel it wil be incremented till 40
        good_image_indx.append(count) ###redundant 
    except TypeError:
        print("The image is not Clear to extract the Embeddings")

    # Display the resulting frame  #####is done wen the embeddings are not clear
        unfit_image_indx.append(count)
        #count+=1
    cv2.imshow('frame',imgf) ######this frame only does all the capturing of 10 frames of each person and then stopping for 15 secs and again starting
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

[]
['stupid']
['stupid', 'swine']


In [15]:
print(user)
print(count)

['stupid', 'swine', '']
31


In [16]:
#########this is for training of knn and svm 

from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score, accuracy_score
a=[]

for jj in range(int(count/10)): ###### the count keeps varying depending on the number of people we live feed it,so here we just divide it  by 10 to get the number of people ,it is divided by 10 because we take 10 images or frames of one person
     a.extend([user[jj] for x in range(10)]) ###this is like having labels for the persons, so first 10 images or first 10 elements of array a will be given name as '0' because the first 10 images are of one personm only ,the next 10 elemnts of a will be given name as '1' beacuse its like the label for the next person 

print(a)
targets = np.array(a)

encoder = LabelEncoder()
encoder.fit(targets)#######the labels
embedded=embedded[0:((int(count/10))*10)] #######this is done beacuse the lenght of embedded can be anything between 0 to 100,depending on the number of people taken during the live feed,so to determine hoe much shuld the lenght be this computstion is done , because the lengt is not a constant
# Numerical encoding of identities
y = encoder.transform(targets)

train_idx = np.arange((int(count/10))*10) % 2 != 0 ###the same reason is here ,we can determine the lenght beforehand ,since it varies dynamically,%2 is just inbuilt
test_idx = np.arange((int(count/10))*10) % 2 == 0

#######the below code is just for training the ml classifiers
X_train = embedded[train_idx]

X_test = embedded[test_idx]

y_train = y[train_idx]
y_test = y[test_idx]

knn = KNeighborsClassifier(n_neighbors=1, metric='euclidean')
svc = LinearSVC()

knn.fit(X_train, y_train)
svc.fit(X_train, y_train)

acc_knn = accuracy_score(y_test, knn.predict(X_test))
acc_svc = accuracy_score(y_test, svc.predict(X_test))

print('KNN accuracy = {0}, SVM accuracy = {1}'.format(acc_knn,acc_svc))

['stupid', 'stupid', 'stupid', 'stupid', 'stupid', 'stupid', 'stupid', 'stupid', 'stupid', 'stupid', 'swine', 'swine', 'swine', 'swine', 'swine', 'swine', 'swine', 'swine', 'swine', 'swine', '', '', '', '', '', '', '', '', '', '']
KNN accuracy = 0.6666666666666666, SVM accuracy = 0.8666666666666667


In [17]:

########a function for the perprocessing for the testing part 
def lol(img):
        embedded1 = np.zeros((100, 128))
        img = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)

        img = align_image(img)
        try:
            img = (img / 255.).astype(np.float32)
        except TypeError:

            print("The image is not Clear to extract the Embeddings")
        else:
            embedded1 = nn4_small2_pretrained.predict(np.expand_dims(img, axis=0))[0]
        testing= np.arange(100)
        example_prediction = knn.predict([embedded1])
        example_identity = encoder.inverse_transform(example_prediction)[0]
        return example_identity
    

In [18]:

import numpy as np
import cv2
from align import AlignDlib
cap = cv2.VideoCapture(0)
#test = load_metadata('F:/face2/face-recognition-master/testimages/')

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# Define the codec and create VideoWriter object.The output is stored in 'outpy.avi' file.
fourcc = cv2.VideoWriter_fourcc('M','J','P','G') 
out = cv2.VideoWriter('F:/face2/face-recognition-master/outputlastnt.avi', fourcc, 20.0, (640, 480)) #not so imp


while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    #cv2.imshow('framezs',frame)
    # Initialize the OpenFace face alignment utility
    try:


        # Detect face and return bounding box
        bb = alignment.getLargestFaceBoundingBox(frame)
        try:
            cv2.rectangle(frame,(bb.left(), bb.top()+bb.height()),(bb.left()+bb.width(), bb.top()),(0,255,0),3)
            #cv2.imshow('framgz',frame) ####the bounding box is put around the face and the coordinates are got from the alignment h    of 
        except:
            #cv2.imshow('framhz',frame)
            out.write(frame)
            print("improper angle or no face")
        # Our operations on the frame come here
        #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        example_identity = lol(frame)
        ###########


        cv2.putText(img = frame, text = "{}".format(example_identity), org = (bb.left(), bb.top()+bb.height()), fontFace = cv2.FONT_HERSHEY_DUPLEX, fontScale = 1, 
                        color = (255, 0, 0))

        #frame = cv2.flip(frame,0)
        ###################
        out.write(frame)
        # Display the resulting frame
        cv2.imshow('frameq',frame) 
    except:
        out.write(frame)
        continue
        #cv2.imshow('framez',frame) q
    #print(('Recogn03zed as {}'.format(example_identity)))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

The image is not Clear to extract the Embeddings
improper angle or no face
The image is not Clear to extract the Embeddings
The image is not Clear to extract the Embeddings
The image is not Clear to extract the Embeddings
The image is not Clear to extract the Embeddings
The image is not Clear to extract the Embeddings
The image is not Clear to extract the Embeddings
The image is not Clear to extract the Embeddings
The image is not Clear to extract the Embeddings


In [19]:
#########for all faces#########